## 연습문제 1
```greeter```의 ```greeting```의 데이터타입을 ```string```에서 ```bytes32```로 변경하세요.
- ```string```에서 ```bytes32```로 변경하면 gas의 차이가 있는지 확인
- 노드에서 배포
- 노드에서 인사를 설정하고 ```setGreeting("Hello World!")```, ```greet()```을 출력하세요.

### 1단계 컨트랙개발

In [1]:
%%writefile src/greeter.sol
pragma solidity ^0.4.0;

contract Greeter{
    bytes32 greeting;
    constructor() public{
        greeting='Hello World!';
    }
    function setGreeting(bytes32 _greeting) public{
        greeting=_greeting;
    }
    function greet() view public returns(bytes32){
        return greeting;
    }
}

Overwriting src/greeter.sol


### 2단계 컴파일

In [2]:
!solc --abi src/greeter.sol


======= src/greeter.sol:Greeter =======
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"_greeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}]


In [3]:
!solc --bin src/greeter.sol


======= src/greeter.sol:Greeter =======
Binary: 
608060405234801561001057600080fd5b507f48656c6c6f20576f726c642100000000000000000000000000000000000000006000816000191690555060ef8061004a6000396000f3006080604052600436106049576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806350513b4f14604e578063cfae321714607c575b600080fd5b348015605957600080fd5b50607a600480360381019080803560001916906020019092919050505060ac565b005b348015608757600080fd5b50608e60ba565b60405180826000191660001916815260200191505060405180910390f35b806000816000191690555050565b600080549050905600a165627a7a7230582099321ee02d8f702f0486c8f57693162d0381ac861bffa9ea638b88d6011cb3960029


In [4]:
!solc --gas src/greeter.sol


======= src/greeter.sol:Greeter =======
Gas estimation:
construction:
   20122 + 47800 = 67922
external:
   greet():	442
   setGreeting(bytes32):	20223


### string, byte32로 출력했을 시, gas의 차이가 있는가?
Greeting 문자열을 string으로 해서 출력했을 때에는, gas비용 산정 시 infinite가 출력되었었다. string은 가변 문자열이기 때문에 bytes32로 코드를 수정하게 되면, gas값이 숫자로 출력되는 것을 확인할 수 있었다. 

### 3단계 배포

In [6]:
%%writefile src/greeterDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
}
var _abiArray=[{"constant":false,"inputs":[{"name":"_greeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="608060405234801561001057600080fd5b507f48656c6c6f20576f726c642100000000000000000000000000000000000000006000816000191690555060ef8061004a6000396000f3006080604052600436106049576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806350513b4f14604e578063cfae321714607c575b600080fd5b348015605957600080fd5b50607a600480360381019080803560001916906020019092919050505060ac565b005b348015608757600080fd5b50608e60ba565b60405180826000191660001916815260200191505060405180910390f35b806000816000191690555050565b600080549050905600a165627a7a7230582099321ee02d8f702f0486c8f57693162d0381ac861bffa9ea638b88d6011cb3960029";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/greeterDeploy.js


In [8]:
!node src/greeterDeploy.js

0x7d842efEf5077092365670b496d11F7B2d0Cd358


### 사용

In [9]:
%%writefile src/greeterUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"_greeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var greeter = new web3.eth.Contract(_abiArray,"0x7d842efEf5077092365670b496d11F7B2d0Cd358");
greeter.methods.setGreeting(web3.utils.fromAscii("Hello World!")).send({from:"0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e",gas:100000});
var sent=web3.utils.fromAscii("Hello!");
greeter.methods.setGreeting(sent);
greeter.methods.greet().call().then(function(value) {console.log(web3.utils.toAscii(value));});

Overwriting src/greeterUse.js


In [10]:
!node src/greeterUse.js

Hello World!                    


## 연습문제 2
어떤 수에 7을 곱하는 스마트 컨트랙을 만든다.
컨트랙은 ```Multiply7```, 함수는:
function multiply(uint input) public pure returns (uint)
컨트랙은 Solidity로, 클라이언트는 노드로 프로그램한다.
- 8을 인자로 넘기면, 7을 곱해서 56을 출력하세요.

### 컨트랙 개발

In [11]:
%%writefile src/Multiply7.sol
pragma solidity ^0.4.0;
contract Multiply7{
    function multiply (uint input) public pure returns(uint) {
        return input*7;
    }
}

Overwriting src/Multiply7.sol


### 컴파일

In [12]:
!solc --abi --bin --gas src/Multiply7.sol


======= src/Multiply7.sol:Multiply7 =======
Gas estimation:
construction:
   87 + 37400 = 37487
external:
   multiply(uint256):	271
Binary: 
608060405234801561001057600080fd5b5060bb8061001f6000396000f300608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063c6888fa1146044575b600080fd5b348015604f57600080fd5b50606c600480360381019080803590602001909291905050506082565b6040518082815260200191505060405180910390f35b60006007820290509190505600a165627a7a72305820cf5147985236c201e297a248bb3be93c27dca120e29720a062634c5d498503ad0029
Contract JSON ABI 
[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}]


### 배포

In [13]:
%%writefile src/Multiply7Deploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
}
var _abiArray=[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}];
var _bin="608060405234801561001057600080fd5b5060bb8061001f6000396000f300608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063c6888fa1146044575b600080fd5b348015604f57600080fd5b50606c600480360381019080803590602001909291905050506082565b6040518082815260200191505060405180910390f35b60006007820290509190505600a165627a7a72305820cf5147985236c201e297a248bb3be93c27dca120e29720a062634c5d498503ad0029";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/Multiply7Deploy.js


In [14]:
!node src/Multiply7Deploy.js

0xF5cB44f22f4F0e7200197b6Ef03Fd3b25400d781


### 사용

In [15]:
%%writefile src/Multiply7Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var abi =[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}];
var addr = "0xF5cB44f22f4F0e7200197b6Ef03Fd3b25400d781";
var counter = new web3.eth.Contract(abi,addr);
counter.methods.multiply(8).call().then(function(str) {console.log(str);});

Overwriting src/Multiply7Use.js


In [16]:
!node src/Multiply7Use.js

56
